In [20]:
from  index_manager import IndexManager
index_manager = IndexManager()
index_manager.delete_index()

Index 'index-orano-investigations' deleted successfully.


In [1]:
from storage_manager import AzureBlobStorageManager


manager = AzureBlobStorageManager() # Initialize the manager
# manager.create_container()  # Create a container
# manager.upload_directory()  # Upload a directory
manager.download_blobs()    # Download all blobs

In [ ]:
from directory_processor import DirectoryProcessor

directory_processor = DirectoryProcessor(manager.get_download_folder_path())
dataset = directory_processor.process_directory()

In [22]:
from llm_models import AzureOpenAIManager

Model = AzureOpenAIManager()
embeddings = Model.create_embeddings()

In [ ]:
from loader_manager import DocumentLoaderManager

loader_manager = DocumentLoaderManager(data=dataset, page_content_column="page_content")

try:
    documents = loader_manager.load_documents_from_dataframe()
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
from splitter_manager import DocumentSplitter

splitter = DocumentSplitter()

try:
    # Split the documents
    splits = splitter.split_documents(documents)
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
from vector_store_manager import VectorStoreManager

vector_store_manager = VectorStoreManager()
vector_test = vector_store_manager.add_documents(splits)


In [ ]:
from retriever_manager import ManagerRetrieve

retriever_manager = ManagerRetrieve()
retriever_manager.create_retriever()
retriever_manager.get_retriever()

In [ ]:
from QA_manager import QuestionAnswerManager

qa_manager = QuestionAnswerManager()
result = qa_manager.ask("DDD des PE de boues sur la fosse 217-02 du DEG ")

result


In [ ]:
from answer_manager import ResultProcessor

answer_manager = ResultProcessor()
answer_manager.create_json_result(result)

In [ ]:
from storage_manager import AzureBlobStorageManager


manager = AzureBlobStorageManager() # Initialize the manager
# manager.create_container()  # Create a container
# manager.upload_directory()  # Upload a directory
manager.download_blobs()    # Download all blobs

from directory_processor import DirectoryProcessor

directory_processor = DirectoryProcessor(manager.get_download_folder_path())
dataset = directory_processor.process_directory()

from llm_models import AzureOpenAIManager

Model = AzureOpenAIManager()
embeddings = Model.create_embeddings()

from loader_manager import DocumentLoaderManager

loader_manager = DocumentLoaderManager(data=dataset, page_content_column="page_content")

try:
    documents = loader_manager.load_documents_from_dataframe()
except Exception as e:
    print(f"An error occurred: {str(e)}")
    
\
from splitter_manager import DocumentSplitter

splitter = DocumentSplitter()

try:
    # Split the documents
    splits = splitter.split_documents(documents)
except Exception as e:
    print(f"An error occurred: {str(e)}")
    
    
from vector_store_manager import VectorStoreManager

vector_store_manager = VectorStoreManager()
vector_test = vector_store_manager.add_documents(splits)


from retriever_manager import ManagerRetrieve

retriever_manager = ManagerRetrieve()
retriever_manager.create_retriever()
retriever_manager.get_retriever()




In [1]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from prompt_manager import PromptManager
from llm_models import AzureOpenAIManager
from retriever_manager import ManagerRetrieve
from answer_manager import ResultProcessor
from storage_manager import AzureBlobStorageManager
from directory_processor import DirectoryProcessor
from loader_manager import DocumentLoaderManager
from splitter_manager import DocumentSplitter
from vector_store_manager import VectorStoreManager

class QuestionAnswerManager:
    def __init__(self):
        """
        Initialize the components necessary for the QuestionAnswerManager, including 
        the language model, retriever, and prompt.
        """
        self.llm = AzureOpenAIManager().create_chat()  # Initialize the Azure OpenAI model
        self.retriever = ManagerRetrieve().get_retriever()  # Initialize the retriever
        self.prompt = PromptManager().get_prompt()  # Get the system prompt template
        
        # Combine the LLM and the prompt into a document processing chain
        self.question_answer_chain = create_stuff_documents_chain(self.llm, self.prompt)
        
        # Create the final chain that retrieves documents and answers questions
        self.chain = create_retrieval_chain(self.retriever, self.question_answer_chain)
    
    def ask(self, user_query):
        """
        Process the user query by invoking the retrieval chain, then process the result 
        to extract relevant information and return a JSON object.
        
        :param user_query: The query from the user.
        :return: A JSON object with the input, answer, and sources.
        """
        # Invoke the chain with the user query
        result = self.chain.invoke({"input": user_query})
        
        # Process the result to create a JSON object
        result = ResultProcessor().create_json_result(result)
        return result
    
    @staticmethod
    def prepare_data():
        """
        A method that handles the preparation of the dataset, including downloading blobs,
        processing directories, splitting documents, and storing vectors.
        """
        # Initialize the Azure Blob Storage Manager and download blobs
        manager = AzureBlobStorageManager()
        manager.download_blobs()  # Download all blobs
        
        # Process the downloaded directory and extract datasets
        directory_processor = DirectoryProcessor(manager.get_download_folder_path())
        dataset = directory_processor.process_directory()
        
        # Create embeddings using the Azure OpenAI model
        model = AzureOpenAIManager()
        embeddings = model.create_embeddings()
        
        # Load documents from the processed dataset
        loader_manager = DocumentLoaderManager(data=dataset, page_content_column="page_content")
        try:
            documents = loader_manager.load_documents_from_dataframe()
        except Exception as e:
            print(f"An error occurred while loading documents: {str(e)}")
            return
        
        # Split the loaded documents into manageable chunks
        splitter = DocumentSplitter()
        try:
            splits = splitter.split_documents(documents)
        except Exception as e:
            print(f"An error occurred while splitting documents: {str(e)}")
            return
        
        # Store the document chunks in a vector store
        vector_store_manager = VectorStoreManager()
        vector_store_manager.add_documents(splits)



In [2]:

# Prepare the data
QuestionAnswerManager.prepare_data()

# Initialize the question-answer manager
qa_manager = QuestionAnswerManager()

# Ask a question
user_question = "Quel est l'objectif de l'investigation sur l’ensemble porte-lame cellule 904?"
response = qa_manager.ask(user_question)

# Print the response
print(response)

File already exists: downloaded_files\DEG\ELH-2018-022986.pdf
File already exists: downloaded_files\DEG\ELH-2019-014855.pdf
File already exists: downloaded_files\DEG\ELH-2019-017580.pdf
File already exists: downloaded_files\DEG\ELH-2019-022027.pdf
File already exists: downloaded_files\DEG\ELH-2019-027845.pdf
File already exists: downloaded_files\DEG\ELH-2020-033633.pdf
File already exists: downloaded_files\DEG\ELH-2020-036076.pdf
File already exists: downloaded_files\DEG\ELH-2021-001129.pdf
File already exists: downloaded_files\DEG\ELH-2021-021517.pdf
File already exists: downloaded_files\HADE\ELH-2019-027755.pdf
File already exists: downloaded_files\HADE\ELH-2019-029034.pdf
File already exists: downloaded_files\HADE\ELH-2019-036670.pdf
File already exists: downloaded_files\HADE\ELH-2019-046650.pdf
File already exists: downloaded_files\HADE\ELH-2019-065388.pdf
File already exists: downloaded_files\HADE\ELH-2019-073706.pdf
File already exists: downloaded_files\HADE\ELH-2020-057124.pdf
F

AttributeError: 'AzureBlobStorageManager' object has no attribute 'get_downloaded_files_path'